In [1]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Model
from keras.models import load_model
from keras.utils import plot_model
from pickle import dump
import os
import time

Using TensorFlow backend.


In [2]:
model = InceptionResNetV2()
model.layers.pop()
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)

In [4]:
model.save('irn2e.h5')

In [3]:
def get_features(file):
    img = load_img(file)
    img = img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img = preprocess_input(img)
    return model.predict(img)

In [4]:
features = dict()
i = 13000

In [ ]:
for filename in (os.listdir('val2014')[i:]):
    features[filename.split('.')[0]] = get_features(os.path.join('val2014',filename))
    i = i + 1
    if i % 1000 == 0:
        dump(features,open('features'+str((i/1000))+'.pkl','wb'))
        features = dict()
        print(i)
    if i % 500 == 0:
        time.sleep(300)

14000
15000
16000
17000
18000
19000
20000


In [1]:
from pickle import load, dump

In [2]:
features = load(open('features.pkl', 'rb'))

# get text description

In [2]:
import pandas as pd
import json
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [3]:
with open('annotations/captions_val2014.json') as f:
   data = json.load(f)

In [7]:
an = pd.DataFrame.from_dict(data['annotations'])

In [8]:
def add_sten(text):
    return 'sq ' + text + ' eq'

In [9]:
an['caption'] = an.apply((lambda x: add_sten(x['caption'])), axis=1)

In [10]:
tok = Tokenizer()
tok.fit_on_texts(an['caption'])

In [11]:
from pickle import dump

In [7]:
vocab_sz = len(tok.word_index)+1

In [10]:
X1, X2, Y = [],[],[]

In [24]:
features["231732"].

array([ 0.43079573,  0.01604018,  0.        , ...,  0.0696895 ,
        0.11172259,  0.16669852], dtype=float32)

In [35]:
X1, X2, Y = [],[],[]

TypeError: Required argument 'shape' (pos 1) not found

In [28]:
cnt = 0
for i in range(len(an)):
    if len(an['caption'][i]) >= 120:
        continue
    img_id = str(an['image_id'][i])
    if img_id not in features:
        continue
    seq = tok.texts_to_sequences([an['caption'][i]])[0]
    for j in range(1,len(seq), 1):
        in_seq, out_seq = seq[:j], seq[j]
        in_seq = pad_sequences([in_seq], maxlen=120)[0]
        out_seq = to_categorical([out_seq], num_classes=vocab_sz)[0]
        X1.append(features[img_id].reshape(1536))
        X2.append(in_seq)
        Y.append(out_seq)
        if len(X1) >= 4095:
            cnt += 1
            x1 = np.array(X1)
            x2 = np.array(X2)
            y = np.array(Y)
            dump([x1, x2, y], open(('D:\\trainfiles\\train.'+str(cnt)+'.pkl'),'wb'))
            del X1, X2, Y, x1, x2, y
            X1, X2, Y = [],[],[]

230


In [36]:
x1,x2, y = load(open(('D:\\trainfiles\\train.'+str(1)+'.pkl'),'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\trainfiles\\train.1.pkl'

In [37]:
x1.shape

(4098, 1536)